In [1]:
#Importing required package
import pandas as pd
import torch
import torch.nn as nn
import

In [4]:
from google.colab import files
uploaded = files.upload()

Saving 100_Unique_QA_Dataset.csv to 100_Unique_QA_Dataset.csv


In [169]:
import pandas as pd

# Load the dataset
df = pd.read_csv("100_Unique_QA_Dataset.csv")
df.head(10)


,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
5,Who painted the Mona Lisa?,Leonardo-da-Vinci
6,What is the square root of 64?,8
7,What is the chemical symbol for gold?,Au
8,Which year did World War II end?,1945
9,What is the longest river in the world?,Nile


In [143]:
# Tokenize a single string
def tokenize(text):
    text = text.lower()
    text = text.replace("?", "").replace("'", "").strip()
    return text.split()

# Vocabulary dictionary with an unknown token
vocab = {"<UNK>": 0}

# Function to update vocabulary from a question-answer row
def build_vocab(row):
    tokenized_question = tokenize(row["question"])
    tokenized_answer = tokenize(row["answer"])
    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)


In [144]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [145]:
len(vocab)

324

In [146]:
#Convert word to neumerical indices
def text_to_indices(text, vocab):

  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
       indexed_text.append(vocab['<UNK>'])
  return indexed_text


In [147]:
text_to_indices("What is prince", vocab)

[1, 2, 0]

In [148]:
import torch
from torch.utils.data import Dataset , DataLoader

In [149]:
class QADataset(Dataset):
  def __init__(self,df, vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self,index):
    neumerical_question = text_to_indices(self.df.iloc[index]["question"],self.vocab)
    neumerical_answer = text_to_indices(self.df.iloc[index]["answer"], self.vocab)
    return torch.tensor(neumerical_question), torch.tensor(neumerical_answer)

In [150]:
dataset = QADataset(df, vocab)

In [151]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [152]:
for question , answer in dataloader:
  print(question, answer)

tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 42, 137,   2, 138,  39, 139]]) tensor([[53]])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([[260]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]]) tensor([[121]])
tensor([[ 10, 308,   3, 309, 310]]) tensor([[311]])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([[194]])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([[154]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([[307]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([[85]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([[317]])
tensor([[ 42, 137,   2,  62,  39,   3, 32

In [153]:
 import torch.nn as nn


In [154]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
        self.rnn = nn.RNN(input_size=50, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, question):
        embedded_question = self.embedding(question)  # (batch, seq_len, 50)
        output, hidden = self.rnn(embedded_question)  # output: (batch, seq_len, 64), hidden: (1, batch, 64)
        output = self.fc(hidden.squeeze(0))           # Use final hidden state (1st dim is num_layers)
        return output                                 # (batch, vocab_size)

In [155]:
#Important variables
learning_rate = 0.001
epochs = 100

In [156]:
model = SimpleRNN(len(vocab))

In [157]:
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [158]:
# Training loop
for epoch in range(epochs):
    total_loss = 0
    for question, answer in dataloader:
        optimizer.zero_grad()

        # Forward pass
        output = model(question)

        # Compute loss
        loss = criterion(output, answer[0])

        # Backpropagation
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 525.2908
Epoch: 2, Loss: 454.4947
Epoch: 3, Loss: 375.0693
Epoch: 4, Loss: 313.3290
Epoch: 5, Loss: 262.4755
Epoch: 6, Loss: 214.3330
Epoch: 7, Loss: 170.4619
Epoch: 8, Loss: 133.1935
Epoch: 9, Loss: 101.7534
Epoch: 10, Loss: 77.9204
Epoch: 11, Loss: 59.7336
Epoch: 12, Loss: 47.0566
Epoch: 13, Loss: 38.0447
Epoch: 14, Loss: 30.7609
Epoch: 15, Loss: 25.5720
Epoch: 16, Loss: 21.4969
Epoch: 17, Loss: 18.2572
Epoch: 18, Loss: 15.7315
Epoch: 19, Loss: 13.5629
Epoch: 20, Loss: 11.8388
Epoch: 21, Loss: 10.4023
Epoch: 22, Loss: 9.1239
Epoch: 23, Loss: 8.0575
Epoch: 24, Loss: 7.1601
Epoch: 25, Loss: 6.4020
Epoch: 26, Loss: 5.7533
Epoch: 27, Loss: 5.2097
Epoch: 28, Loss: 4.7132
Epoch: 29, Loss: 4.2836
Epoch: 30, Loss: 3.9121
Epoch: 31, Loss: 3.5738
Epoch: 32, Loss: 3.2892
Epoch: 33, Loss: 3.0318
Epoch: 34, Loss: 2.8078
Epoch: 35, Loss: 2.5863
Epoch: 36, Loss: 2.3950
Epoch: 37, Loss: 2.2358
Epoch: 38, Loss: 2.0722
Epoch: 39, Loss: 1.9345
Epoch: 40, Loss: 1.8055
Epoch: 41, Loss: 1.

In [159]:
def predict(model, question, threshold= 0.5):
  #Convert to number
  neumerical_question=text_to_indices(question, vocab)


  #tensor
  qes_tensor = torch.tensor(neumerical_question).unsqueeze(0)

  #send to model
  output = model(qes_tensor)


  #convert logits to probabilities
  probs = torch.nn.functional.softmax(output, dim=1)

  #find index of max probablity
  value, index=torch.max(probs, dim=1)

  if value < threshold:
    print("I dont know?")

  ans =list(vocab.keys())[index]
  print(ans)


In [183]:
ques = input("Enter question:-")
print(predict(model, ques))

Enter question:-Who wrote 'To Kill a Mockingbird'?	
harper-lee
None


In [165]:
list(vocab.keys())[10]

'who'